In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
filename = "N2H2_VVdata_3variables.xlsx"
df = pd.read_excel(filename)
debug = False
print(df.columns)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

x = df[['v', 'w', 'T(K)']].values
#use logscakle 
y = np.log10(df[['k(cm^3/s)']].values)

scalerx = MinMaxScaler()
scalerx.fit(x)
x_s = scalerx.transform(x)

#print(x_s[:,0])
#print(x[:,0])

vmap_toreal = {}

for i, vn in enumerate(x_s[:,0]):
    vmap_toreal[vn] = x[i,0]

print("V map: ")
for a in vmap_toreal:
    print("%4.2f --> %3d"%(a, vmap_toreal[a]))

vset = set(x_s[:,0])
wset = set(x_s[:,1])
tset = set(x_s[:,2])

scalery = MinMaxScaler()
scalery.fit(y)
y_s = scalery.transform(y)

if debug:
    for i, ys in enumerate(y_s):
        print(ys, y[i])
    for i, xs in enumerate(x_s):
        print(xs, x[i])

In [ ]:
from tensorflow import keras
import tensorflow as tf

import tensorflow.keras.optimizers as tko
import tensorflow.keras.activations as tka
import tensorflow.keras.losses as tkl
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn import metrics

import commonmodules as cm

In [ ]:
# chek data
#for w in wset:
#    train_x, test_x, train_y, test_y = cm.test_train_split (1, [w], x_s, y_s)
#
#    cm.plotfull3dcurve (1, test_x, test_y)

In [ ]:
ofp = open("vremoved.csv", "w")

modelshape = [32, 32, 32, 32]
epochs = 20
batch_size = 100

thefirst = True

print (" v Removed , Test MSE , Test R2 , Train MSE , Train R2")
print (" v Removed , Test MSE , Test R2 , Train MSE , Train R2", file=ofp)
for v in vset:
    train_x, test_x, train_y, test_y = cm.test_train_split (0, [v], x_s, y_s)

    if thefirst:
        model = cm.buildmodel(modelshape)
        history = model.fit(train_x, train_y, epochs=epochs,  batch_size=batch_size, \
            verbose=0)
        thefirst = False

    model = cm.buildmodel(modelshape)
    history = model.fit(train_x, train_y, epochs=epochs,  batch_size=batch_size, \
        verbose=0)
    
    ofptest = open("vremoved_"+str(vmap_toreal[v])+"_test.csv", "w")
    print (" v , w , T , y , y_pred ", file=ofptest)
    test_x_sp = scalerx.inverse_transform(test_x)
    pred_y = model.predict(test_x, verbose=0)
    pred_y_sb = scalery.inverse_transform(pred_y)
    test_y_sb = scalery.inverse_transform(test_y)
    for i, yt in enumerate(test_y_sb):
         print (" %3d , %3d , %6d , %10.8e , %10.8e  "%(test_x_sp[i,0], 
                                             test_x_sp[i,1],
                                             test_x_sp[i,2],
                                             yt,
                                             pred_y_sb[i]), file=ofptest, flush=True)
    #plt.scatter(test_y_sb, pred_y_sb)
    #plt.show()
    testmse = metrics.mean_absolute_error(test_y_sb, pred_y_sb)
    testr2 = metrics.r2_score(test_y_sb, pred_y_sb)
    ofptest.close()

    ofptrain = open("vremoved_"+str(vmap_toreal[v])+"_train.csv", "w")
    print (" v , w , T , y , y_pred  ", file=ofptrain)
    pred_y = model.predict(train_x, verbose=0)
    pred_y_sb = scalery.inverse_transform(pred_y)
    train_y_sb = scalery.inverse_transform(train_y)
    train_x_sp = scalerx.inverse_transform(train_x)
    for i, yt in enumerate(train_y_sb):
         print (" %3d , %3d , %6d , %10.8e , %10.8e  "%(train_x_sp[i,0], 
                                             train_x_sp[i,1],
                                             train_x_sp[i,2],
                                             yt,
                                             pred_y_sb[i]), file=ofptrain, flush=True)
    #plt.scatter(train_y_sb, pred_y_sb)
    #plt.show()
    trainmse = metrics.mean_absolute_error(train_y_sb, pred_y_sb)
    trainr2 = metrics.r2_score(train_y_sb, pred_y_sb)
    ofptrain.close()
    
    print("%3d , %10.6e , %10.6f , %10.6e , %10.6f"%(vmap_toreal[v], testmse, testr2, \
                                                       trainmse,  trainr2))
    
    print("%3d , %10.6e , %10.6f , %10.6e , %10.6f"%(vmap_toreal[v], testmse, testr2, \
                                                       trainmse,  trainr2), file=ofp)
    
ofp.close()

In [ ]:
vlist = list(vset)

vset_torm = []

vtoremove = []
for i in range(1,len(vlist),2):
    vtoremove.append(vlist[i])
vset_torm.append(vtoremove)

vtoremove = []
for i in range(0,len(vlist),2):
    vtoremove.append(vlist[i])
vset_torm.append(vtoremove)

vtoremove = []
for i in range(1,len(vlist),3):
    vtoremove.append(vlist[i])
    if (i+1 < len(vlist)):
        vtoremove.append(vlist[i+1])
vset_torm.append(vtoremove)

vtoremove = []
for i in range(0,len(vlist),3):
    vtoremove.append(vlist[i])
    if (i+1 < len(vlist)):
        vtoremove.append(vlist[i+1])
vset_torm.append(vtoremove)

vtoremove = []
for i in range(1,len(vlist),4):
    vtoremove.append(vlist[i])
    if (i+1 < len(vlist)):
        vtoremove.append(vlist[i+1])
    if (i+2 < len(vlist)):
        vtoremove.append(vlist[i+2])
vset_torm.append(vtoremove)

vtoremove = []
for i in range(0,len(vlist),4):
    vtoremove.append(vlist[i])
    if (i+1 < len(vlist)):
        vtoremove.append(vlist[i+1])
    if (i+2 < len(vlist)):
        vtoremove.append(vlist[i+2])
vset_torm.append(vtoremove)

thefirst = True

ofp = open("vsetremoved.csv", "w")

modelshape = [32, 32, 32, 32]
epochs = 20
batch_size = 100

print (" vset Removed , Test MSE , Test R2 , Train MSE , Train R2")
print (" vset Removed , Test MSE , Test R2 , Train MSE , Train R2", file=ofp)
for setid, v in enumerate(vset_torm):

    train_x, test_x, train_y, test_y = cm.test_train_split (0, v, x_s, y_s)

    v_sp = []
    for val in v:
        v_sp.append(vmap_toreal[val]) 
        
    if thefirst:
        model = cm.buildmodel(modelshape)
        history = model.fit(train_x, train_y, epochs=epochs,  batch_size=batch_size, \
            verbose=0)
        thefirst = False

    model = cm.buildmodel(modelshape)
    history = model.fit(train_x, train_y, epochs=epochs,  batch_size=batch_size, \
        verbose=0)

    ofptest = open("vremoved_set"+str(setid+1)+"_test.csv", "w")
    print (" v , w , T , y , y_pred", file=ofptest)
    pred_y = model.predict(test_x, verbose=0)
    test_x_sp = scalerx.inverse_transform(test_x)
    pred_y_sb = scalery.inverse_transform(pred_y)
    test_y_sb = scalery.inverse_transform(test_y)
    for i, yt in enumerate(test_y_sb):
        print (" %3d , %3d , %6d , %10.8e , %10.8e  "%(test_x_sp[i,0], 
                                             test_x_sp[i,1],
                                             test_x_sp[i,2],
                                             yt,
                                             pred_y_sb[i]), file=ofptest, flush=True)
    #plt.scatter(test_y_sb, pred_y_sb)
    #plt.show()
    testmse = metrics.mean_absolute_error(test_y_sb, pred_y_sb)
    testr2 = metrics.r2_score(test_y_sb, pred_y_sb)
    ofptest.close()

    ofptrain = open("vremoved_set"+str(setid+1)+"_train.csv", "w")
    print (" v , w , T , y , y_pred  ", file=ofptrain)
    pred_y = model.predict(train_x, verbose=0)
    pred_y_sb = scalery.inverse_transform(pred_y)
    train_y_sb = scalery.inverse_transform(train_y)
    train_x_sp = scalerx.inverse_transform(train_x)
    for i, yt in enumerate(train_y_sb):
         print (" %3d , %3d , %6d , %10.8e , %10.8e  "%(train_x_sp[i,0], 
                                             train_x_sp[i,1],
                                             train_x_sp[i,2],
                                             yt,
                                             pred_y_sb[i]), file=ofptrain, flush=True)
    #plt.scatter(train_y_sb, pred_y_sb)
    #plt.show()
    trainmse = metrics.mean_absolute_error(train_y_sb, pred_y_sb)
    trainr2 = metrics.r2_score(train_y_sb, pred_y_sb)
    ofptrain.close()
        
    print("%s , %10.6e , %10.6f , %10.6e , %10.6f"%(v_sp, testmse, testr2, \
                                                        trainmse,  trainr2), flush=True)
    print("%s , %10.6e , %10.6f , %10.6e , %10.6f"%(v_sp, testmse, testr2, \
                                                        trainmse,  trainr2), file=ofp, flush=True)
    
ofp.close()

In [ ]:
"""

ofp = open("perc.csv", "w")

print (" Perc. Split , Test MSE , Test R2 , Train MSE , Train R2")
print (" Perc. Split , Test MSE , Test R2 , Train MSE , Train R2", file=ofp)
for perc in [0.05, 0.10, 0.25, 0.30, 0.50]:
    train_x, test_x, train_y, test_y = train_test_split(x_s, y_s, \
                    test_size=perc, random_state=42)

    modelshape = [64, 64, 64]
    epochs = 20
    batch_size = 50

    model = cm.buildmodel(modelshape)
    history = model.fit(train_x, train_y, epochs=epochs,  batch_size=batch_size, \
        verbose=0)

    pred_y = model.predict(test_x, verbose=0)
    #to scale back y
    #pred_y_sb = scalery.inverse_transform(pred_y)
    #y_sb = scalery.inverse_transform(test_y)
    #plt.scatter(y_sb, pred_y_sb)
    #plt.show()
    testmse = metrics.mean_absolute_error(test_y, pred_y)
    testr2 = metrics.r2_score(test_y, pred_y)

    pred_y = model.predict(train_x, verbose=0)
    trainmse = metrics.mean_absolute_error(train_y, pred_y)
    trainr2 = metrics.r2_score(train_y, pred_y)

    print("%5.2f , %10.6f , %10.6f , %10.6f , %10.6f"%(perc, testmse, testr2, \
                                                       trainmse,  trainr2))
    print("%5.2f , %10.6f , %10.6f , %10.6f , %10.6f"%(perc, testmse, testr2, \
                                                       trainmse,  trainr2), file=ofp)
    
ofp.close()
"""

In [ ]:
"""
ofp = open("wremoved.csv", "w")

thefirst = True

print (" w Removed , Test MSE , Test R2 , Train MSE , Train R2")
print (" w Removed , Test MSE , Test R2 , Train MSE , Train R2", file=ofp)
for w in wset:
    train_x, test_x, train_y, test_y = cm.test_train_split (1, [w], x_s, y_s)

    modelshape = [64, 64, 64]
    epochs = 20
    batch_size = 50

    if thefirst:
        model = cm.buildmodel(modelshape)
        history = model.fit(train_x, train_y, epochs=epochs,  batch_size=batch_size, \
            verbose=0)
        thefirst = False

    model = cm.buildmodel(modelshape)
    history = model.fit(train_x, train_y, epochs=epochs,  batch_size=batch_size, \
        verbose=0)

    pred_y = model.predict(test_x, verbose=0)
    #to scale back y
    #pred_y_sb = scalery.inverse_transform(pred_y)
    #y_sb = scalery.inverse_transform(test_y)
    #plt.scatter(y_sb, pred_y_sb)
    #plt.show()
    testmse = metrics.mean_absolute_error(test_y, pred_y)
    testr2 = metrics.r2_score(test_y, pred_y)

    pred_y = model.predict(train_x, verbose=0)
    trainmse = metrics.mean_absolute_error(train_y, pred_y)
    trainr2 = metrics.r2_score(train_y, pred_y)

    print("%5.2f , %10.6f , %10.6f , %10.6f , %10.6f"%(w, testmse, testr2, \
                                                       trainmse,  trainr2))
    
    print("%5.2f , %10.6f , %10.6f , %10.6f , %10.6f"%(w, testmse, testr2, \
                                                       trainmse,  trainr2), file=ofp)
    
ofp.close()
"""

In [ ]:
"""
ofp = open("tremoved.csv", "w")

thefirst = True

print (" T Removed , Test MSE , Test R2 , Train MSE , Train R2")
print (" T Removed , Test MSE , Test R2 , Train MSE , Train R2", file=ofp)
for t in tset:
    train_x, test_x, train_y, test_y = cm.test_train_split (2, [t], x_s, y_s)

    modelshape = [64, 64, 64]
    epochs = 20
    batch_size = 50

    if thefirst:
        model = cm.buildmodel(modelshape)
        history = model.fit(train_x, train_y, epochs=epochs,  batch_size=batch_size, \
            verbose=0)
        thefirst = False

    model = cm.buildmodel(modelshape)
    history = model.fit(train_x, train_y, epochs=epochs,  batch_size=batch_size, \
        verbose=0)

    pred_y = model.predict(test_x, verbose=0)
    #to scale back y
    #pred_y_sb = scalery.inverse_transform(pred_y)
    #y_sb = scalery.inverse_transform(test_y)
    #plt.scatter(y_sb, pred_y_sb)
    #plt.show()
    testmse = metrics.mean_absolute_error(test_y, pred_y)
    testr2 = metrics.r2_score(test_y, pred_y)

    pred_y = model.predict(train_x, verbose=0)
    trainmse = metrics.mean_absolute_error(train_y, pred_y)
    trainr2 = metrics.r2_score(train_y, pred_y)

    print("%5.2f , %10.6f , %10.6f , %10.6f , %10.6f"%(t, testmse, testr2, \
                                                       trainmse,  trainr2))
    print("%5.2f , %10.6f , %10.6f , %10.6f , %10.6f"%(t, testmse, testr2, \
                                                       trainmse,  trainr2), file=ofp)
    
ofp.close()
"""